<a href="https://colab.research.google.com/github/Vinh-thuy/data_valorization/blob/main/demo/prompt_generator_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nlpcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#@title Connexion Google Drive {display-mode: "form"}

from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

%cd /content/gdrive/MyDrive/P4T/google_colab/demo/


Mounted at /content/gdrive/
/content/gdrive/MyDrive/P4T/google_colab/demo


In [3]:
import pandas as pd 
review_summary=pd.read_csv('/content/gdrive/MyDrive/P4T/google_colab/demo/review_summary.csv')

In [4]:
#@title Construction de la connexion à la base PostgreSQL {display-mode: "form"}

from postgres import postgresQuery
from utilities import DB_HOSTNAME,DB_PORT, DB_NAME, DB_USER, DB_PWD, PG_COL_TO_CLEAN_TABLE_RESTAURANT
pg = postgresQuery(DB_HOSTNAME, DB_PORT, DB_NAME, DB_USER, DB_PWD)

In [5]:
#@title Récupération des tables : restaurant {display-mode: "form"}
city = 'Paris'

sql_query_1 = "select *  from restaurant r2  where r2.adress_city  = '" + city + "'"
sql_query_2 = "select r1.*, r2.review_id as review_id, r2.reviews_data as reviews_data from restaurant r1, review r2  where r1.restaurant_id = r2.restaurant_id and r1.adress_city  = '" + city + "'"


liste_restaurant = pg.query_db(sql_query_1)
liste_restaurant_review = pg.query_db(sql_query_2)


liste_restaurant['created_at'] = pd.to_datetime(liste_restaurant['created_at']).dt.normalize()

liste_restaurant_review['created_at'] = pd.to_datetime(liste_restaurant_review['created_at']).dt.normalize()

In [6]:
mapping_restaurant_review = liste_restaurant_review[['review_id', 'restaurant_id', 'criteria']]
mapping_restaurant_review_criteriaNotNull = mapping_restaurant_review[mapping_restaurant_review.review_id.isin(mapping_restaurant_review.dropna(how='any',axis=0).review_id)]

In [7]:
#@title Load GPT Extraction NLP CLOUD depuis Google Drive : review_summary_split {display-mode: "form"}

import pandas as pd 
review_summary_split=pd.read_csv('/content/gdrive/MyDrive/P4T/google_colab/demo/review_summary_split.csv')  
review_summary_split.head(1)

,Unnamed: 0,keyphrase_extract_review,review_id,review_category,created_at
0,0,"['1', ' Pâtes fraîches délicieuses et savoureu...",66347,review_positif,2023-01-10 09:04:17


In [320]:
#@title Fonction : clean_text >> NON UTILISE {display-mode: "form"}

def clean_text(text):
  import re

  #text = re.sub(r'\n\d', ' ', text).replace("\nspoilers","").replace("\n","")
  cleanedtext = []
  for item in nlp(text):
      if not item.is_stop:
          cleanedtext.append(item.text)
  text = ' '.join(cleanedtext)

  #text = re.sub(r"\'", ' ', text).replace("\nspoilers","").replace("\n","")
  #text = re.sub(r" . ", ' ', text).replace("","").replace("","")
  #text = re.sub(r",. ", ', ', text).replace("","").replace("","")

  lst = text.split(",") 
  
  return lst


In [8]:
df = review_summary
df2 = mapping_restaurant_review_criteriaNotNull
df3 = df.merge(df2, on='review_id', how='left')

#df4 = pd.DataFrame(df3.restaurant_id.unique(), columns=['restaurant_id'])
#df4['criteria_list'] = "[]"

df5 = df3[['restaurant_id', 'criteria']]
df5 = df5.drop_duplicates()


# Version en mode LIST
"""
for (id) in enumerate(df5.restaurant_id):
  kpe_list = []

  for keyphrase_extract_review in zip(df3[df3.restaurant_id == id[1]].keyphrase_extract_review):
    for elt in enumerate(keyphrase_extract_review[0].split(",")):
      kpe_list.append(elt)
  df5.loc[df5["restaurant_id"] == id[1], "kpe_list"] = [kpe_list]
"""

# Version en mode STR
for (id) in enumerate(df5.restaurant_id):
  kpe_review =""
  for keyphrase_extract_review in zip(df3[df3.restaurant_id == id[1]].keyphrase_extract_review):
    for elt in enumerate(keyphrase_extract_review[0].split(",")):
      kpe_review = kpe_review + (elt[1])
  df5.loc[df5["restaurant_id"] == id[1], "kpe_review"] = kpe_review


# On ajoute une entete par rubrique : ectraction Review & Criteres du restaurant
df5["kpe_review"] = "avis positif du restaurant :  " + df5["kpe_review"]
df5["criteria"] = "service disponibles du restaurant :  " + df5["criteria"]


# Création du prompt pour GPT
for (id) in enumerate(df5.restaurant_id):
  kpe_review = df5[df5["restaurant_id"] == id[1]].kpe_review.values[0]
  criteria = df5[df5["restaurant_id"] == id[1]].criteria.values[0] 
  df5.loc[df5["restaurant_id"] == id[1], "prompt_GPT"] = kpe_review + " ----- " + criteria


In [34]:
df5

,restaurant_id,criteria,kpe_review,prompt_GPT
0,835,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Le service ...,avis positif du restaurant : ['1. Le service ...
1,792,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Service pro...,avis positif du restaurant : ['1. Service pro...
5,774,service disponibles du restaurant : ['Service...,avis positif du restaurant : ['1. Prix raison...,avis positif du restaurant : ['1. Prix raison...
6,977,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Garçons sym...,avis positif du restaurant : ['1. Garçons sym...
8,744,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Plats varié...,avis positif du restaurant : ['1. Plats varié...
11,805,service disponibles du restaurant : ['Service...,avis positif du restaurant : ['1. Produits de...,avis positif du restaurant : ['1. Produits de...
13,790,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Service trè...,avis positif du restaurant : ['1. Service trè...
16,819,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1' ' Ambiance ...,avis positif du restaurant : ['1' ' Ambiance ...
21,925,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Une cuisine...,avis positif du restaurant : ['1. Une cuisine...
24,927,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. La pizza es...,avis positif du restaurant : ['1. La pizza es...


In [43]:
# RECHERCHE COMPLEXE DE L'UTILISATEUR
MB_question = """
dis moi avec les avis et les services de ce restaurant si ils répondent à la recherche qui suit  : 
- le restaurant doit absolument proposer des pates fraiches
- le restaurant doit etre pas trop cher

- sans générer de nouveau texte
- de facon completement objectif
"""

In [44]:
import nlpcloud
client = nlpcloud.Client("finetuned-gpt-neox-20b", "1cc13406650e4690596159d13c312538b2e8346f", gpu=True, lang="en")

In [46]:
res = client.question(MB_question,texte)

In [45]:
texte = df5[df5["restaurant_id"] == 985].prompt_GPT.values[0]

texte

"avis positif du restaurant :  ['1 ' ' Cuisine authentique et délicieuse ' '    ' ' Large gamme de plats ' '  ' ' Vins grecs à prix équitables ' '  ' ' Ambiance familiale ' '  ' ' Service rapide et attentionné.'] ----- service disponibles du restaurant :  ['Paiements : Paiements mobiles NFC', 'Offre : Vin', 'Paiements : Cartes de paiement', 'Offre : Produits sains', 'Offre : Petites portions à partager', 'Services de restauration : Places assises', 'Ambiance : Décontracté', 'Services disponibles : Repas sur place', 'Offre : Convient aux végétariens', 'Paiements : Cartes de crédit', 'Offre : Alcools', 'Offre : Bière', 'Services de restauration : Desserts', 'Ambiance : Chaleureux', 'Services de restauration : Traiteur', 'Services de restauration : Déjeuner', 'Services disponibles : Vente à emporter', 'Services disponibles : Livraison', 'Offre : Cafés']"

In [39]:
df5[df5["restaurant_id"] == 985].prompt_GPT.values[0]

"avis positif du restaurant :  ['1 ' ' Cuisine authentique et délicieuse ' '    ' ' Large gamme de plats ' '  ' ' Vins grecs à prix équitables ' '  ' ' Ambiance familiale ' '  ' ' Service rapide et attentionné.'] ----- service disponibles du restaurant :  ['Paiements : Paiements mobiles NFC', 'Offre : Vin', 'Paiements : Cartes de paiement', 'Offre : Produits sains', 'Offre : Petites portions à partager', 'Services de restauration : Places assises', 'Ambiance : Décontracté', 'Services disponibles : Repas sur place', 'Offre : Convient aux végétariens', 'Paiements : Cartes de crédit', 'Offre : Alcools', 'Offre : Bière', 'Services de restauration : Desserts', 'Ambiance : Chaleureux', 'Services de restauration : Traiteur', 'Services de restauration : Déjeuner', 'Services disponibles : Vente à emporter', 'Services disponibles : Livraison', 'Offre : Cafés']"

In [ ]:
# Création du prompt pour GPT
for (id) in enumerate(df5.restaurant_id):
  texte = df5[df5["restaurant_id"] == id[1]].prompt_GPT.values[0]
  res = client.question(MB_question,texte)
  df5.loc[df5["restaurant_id"] == id[1], "question_GPT"] = res['answer']

In [63]:
# Export des résultats dans Google Drive

df5.to_csv('intention_result.csv')
!cp data.csv "/content/gdrive/MyDrive/P4T/google_colab/demo/"

cp: 'data.csv' and '/content/gdrive/MyDrive/P4T/google_colab/demo/data.csv' are the same file


In [62]:
df5

,restaurant_id,criteria,kpe_review,prompt_GPT,question_GPT
0,835,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Le service ...,avis positif du restaurant : ['1. Le service ...,NaN
1,792,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Service pro...,avis positif du restaurant : ['1. Service pro...,NaN
5,774,service disponibles du restaurant : ['Service...,avis positif du restaurant : ['1. Prix raison...,avis positif du restaurant : ['1. Prix raison...,NaN
6,977,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Garçons sym...,avis positif du restaurant : ['1. Garçons sym...,NaN
8,744,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Plats varié...,avis positif du restaurant : ['1. Plats varié...,NaN
11,805,service disponibles du restaurant : ['Service...,avis positif du restaurant : ['1. Produits de...,avis positif du restaurant : ['1. Produits de...,NaN
13,790,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Service trè...,avis positif du restaurant : ['1. Service trè...,NaN
16,819,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1' ' Ambiance ...,avis positif du restaurant : ['1' ' Ambiance ...,NaN
21,925,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. Une cuisine...,avis positif du restaurant : ['1. Une cuisine...,NaN
24,927,service disponibles du restaurant : ['Offre :...,avis positif du restaurant : ['1. La pizza es...,avis positif du restaurant : ['1. La pizza es...,NaN
